In [14]:
import pandas as pd

In [15]:
data=[]
with open('log/FIX.4.2-FIXTEST-CSCII2.messages.current.log', 'r') as file:
    for line in file:
        fields=line.rstrip().split(chr(1))
        record={}
        for item in fields:
            if item:
                key, value=item.split('=')
                record[key]=value
        data.append(record)


In [16]:
orders_send=[item for item in data if item.get('35')=='D']
orders_recv=[item for item in data if item.get('39')=='0']
cancel_send=[item for item in data if item.get('35')=='F']
cancel_recv=[item for item in data if item.get('39')=='4']

In [17]:
len(orders_send), len(orders_recv), len(cancel_send), len(cancel_recv)

(10, 10, 10, 10)

In [18]:
orders_send_dt=[{'id': item['11'], 'dt':key[:-4]} for item in orders_send for key in item if key.startswith('2023')]
orders_recv_dt=[{'id': item['11'], 'dt':key[:-4]} for item in orders_recv for key in item if key.startswith('2023')]
cancel_send_dt=[{'id': item['11'], 'dt':key[:-4], 'origin_id':item['41']} for item in cancel_send for key in item if key.startswith('2023')]
cancel_recv_dt=[{'id': item['11'], 'dt':key[:-4], 'origin_id':item['41']} for item in cancel_recv for key in item if key.startswith('2023')]

In [19]:
df_order_send=pd.DataFrame(orders_send_dt)
df_order_recv=pd.DataFrame(orders_recv_dt)
df_cancel_send=pd.DataFrame(cancel_send_dt)
df_cancel_recv=pd.DataFrame(cancel_recv_dt)

In [20]:
df_order=pd.merge(df_order_send, df_order_recv, on='id', how='inner', suffixes=('_send', '_recv'))
df_cancel=pd.merge(df_cancel_send, df_cancel_recv, on=['id', 'origin_id'], how='inner',suffixes=('_send', '_recv'))

In [21]:
df_order.dt_send=pd.to_datetime(df_order.dt_send)
df_order.dt_recv=pd.to_datetime(df_order.dt_recv)
df_cancel.dt_send=pd.to_datetime(df_cancel.dt_send)
df_cancel.dt_recv=pd.to_datetime(df_cancel.dt_recv)

In [22]:
df_order['latency']=(df_order.dt_recv-df_order.dt_send).dt.total_seconds()*1000
df_cancel['latency']=(df_cancel.dt_recv-df_cancel.dt_send).dt.total_seconds()*1000

In [23]:
# df_order.sort_values(by=['dt_send',]).to_csv('order-local.csv', index=False, sep='\t')

In [24]:
# df_cancel.sort_values(by=['dt_send',]).to_csv('cancel-local.csv', index=False, sep='\t')

In [27]:
pd.merge(left=df_order, right=df_cancel,left_on='id', right_on='origin_id', how='inner', suffixes=('_order', '_cancel'))

,id_order,dt_send_order,dt_recv_order,latency_order,id_cancel,dt_send_cancel,origin_id,dt_recv_cancel,latency_cancel
0,1869009,2023-10-10 01:58:18.082363,2023-10-10 01:58:18.218924,136.561,1035306,2023-10-10 01:58:19.084431,1869009,2023-10-10 01:58:19.714312,629.881
1,1068365,2023-10-10 01:58:20.087829,2023-10-10 01:58:20.205826,117.997,1206993,2023-10-10 01:58:21.089307,1068365,2023-10-10 01:58:21.702437,613.130
2,1121998,2023-10-10 01:58:22.090637,2023-10-10 01:58:22.210166,119.529,1580468,2023-10-10 01:58:23.092774,1121998,2023-10-10 01:58:23.687744,594.970
3,1989592,2023-10-10 01:58:24.094552,2023-10-10 01:58:24.237660,143.108,1653929,2023-10-10 01:58:25.096514,1989592,2023-10-10 01:58:25.725366,628.852
4,1653581,2023-10-10 01:58:26.098241,2023-10-10 01:58:26.217941,119.700,1725045,2023-10-10 01:58:27.099642,1653581,2023-10-10 01:58:27.712634,612.992
5,1680286,2023-10-10 01:58:28.100974,2023-10-10 01:58:28.224406,123.432,1376940,2023-10-10 01:58:29.103097,1680286,2023-10-10 01:58:29.716556,613.459
6,1579133,2023-10-10 01:58:30.104786,2023-10-10 01:58:30.241198,136.412,1401585,2023-10-10 01:58:31.106421,1579133,2023-10-10 01:58:31.732532,626.111
7,1804919,2023-10-10 01:58:32.108057,2023-10-10 01:58:32.226423,118.366,1610148,2023-10-10 01:58:33.109847,1804919,2023-10-10 01:58:33.720431,610.584
8,1578481,2023-10-10 01:58:34.111120,2023-10-10 01:58:34.230373,119.253,1751920,2023-10-10 01:58:35.112587,1578481,2023-10-10 01:58:35.706783,594.196
9,1261295,2023-10-10 01:58:36.113756,2023-10-10 01:58:36.250802,137.046,1293013,2023-10-10 01:58:37.115929,1261295,2023-10-10 01:58:37.693074,577.145
